In [10]:
from embeddings import compute_umap, compute_tsne
import tifffile as tif
import numpy as np
import pandas as pd
import utils
import visualizer
import utils
from sklearn.cluster import KMeans

data_path = '../data/'
images_path = data_path + 'images/'


In [11]:
# read images and convert to numpy array
data_path = '../data/'
images_path = data_path + 'images/'

images_list = utils.list_files(images_path)

images = []
for image in images_list:
    images.append(tif.imread(images_path + image))
images = np.array(images)
print('Shape of images:', images.shape)

# convert to nx(d1*d2) array
images = images.reshape(images.shape[0], -1)
print('Shape of images:', images.shape)

Shape of images: (1000, 28, 28)
Shape of images: (1000, 784)


In [9]:
# Compute UMAP

umap_embeddings = compute_umap(images, n_components=10, n_neighbors=15, min_dist=0.1, metric='euclidean')

# save all in a pandas dataframe
df = pd.DataFrame()
df['image'] = images_list
df['umap_embedding_1'] = umap_embeddings[:, 0]
df['umap_embedding_2'] = umap_embeddings[:, 1]


/Users/soitu/anaconda3/envs/cellpose3/lib/python3.8/site-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.

OMP: Info #273: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


KeyboardInterrupt: 

In [20]:
# add columns for the visualizer: x pixels, y pixels, url of the image in base64, some clustering

df['url'] = np.nan

for i, row in df.iterrows():
    df.at[i, 'url'] = utils.encode_image_to_base64(images_path + row['image'])

# k-means clustering on the reduced_embeddings
kmeans = KMeans(n_clusters=10, random_state=0).fit(df[['umap_embedding_1', 'umap_embedding_2']])
df['k_means'] = kmeans.labels_
df.to_csv(data_path + 'embeddings.csv', index=False)

/Users/soitu/anaconda3/envs/cellpose3/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [12]:
visualizer.viz_umap_plot(df_path=data_path + 'embeddings.csv', embeddings_col_0='umap_embedding_1', embeddings_col_1='umap_embedding_2', url_col='url', display_columns = ['image', 'k_means', 'x_px'], cluster_col= 'k_means', display_fraction=0.9)